# Import Libraries

In [ ]:
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import keras
from keras.layers import Conv2D, BatchNormalization, PReLU, MaxPool2D, Dropout, Dense, Flatten
from keras.regularizers import L1L2
from keras.utils import to_categorical

print("Available GPUs:", tf.config.list_physical_devices('GPU'))
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
tf.config.set_soft_device_placement(True)
# GPU 메모리 증가 허용 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
import tf2onnx

## Data Load

In [ ]:
mpii_path = "../data"

# 전체 참가자 클래스에 대하여 Train/Valid 균등 분류
within_id_data = np.load(os.path.join(mpii_path, "within_ids.npy"))
within_hps_data = np.load(os.path.join(mpii_path, "within_2d_hps.npy"))
within_img_data = np.load(os.path.join(mpii_path, "within_images.npy"))
within_gzs_data = np.load(os.path.join(mpii_path, "within_2d_gazes.npy"))

print(f"within_id_data : {within_id_data.shape}")
print(f"within_hps_data : {within_hps_data.shape}")
print(f"within_img_data : {within_img_data.shape}")
print(f"within_gzs_data : {within_gzs_data.shape}\n")

# ID 데이터를 정수 인덱스로 변환
unique_ids, id_indices = np.unique(within_id_data, return_inverse=True)
n_clusters = len(unique_ids)  # 클러스터(참가자) 수

# 원-핫 인코딩으로 클러스터 정보 변환
id_one_hot = to_categorical(id_indices).reshape(within_id_data.shape[0], within_id_data.shape[1], -1)
id_one_hot = id_one_hot.reshape(-1, n_clusters)  # 샘플 수, 클러스터 수

# 클러스터 데이터 형태 변환
ids_data = within_id_data.reshape(-1,1)
l_enc = LabelEncoder()
ids_enc_data = l_enc.fit_transform(ids_data)

# 이미지 데이터 정규화 및 형태 변환
normalized_img_data = within_img_data / 255.0
normalized_img_data = normalized_img_data.reshape(-1, 36, 60, 1)  # 샘플 수, 높이, 너비, 채널

# 헤드포즈 데이터 형태 변환
hps_data = within_hps_data.reshape(-1, 2)  # 샘플 수, 헤드포즈 차원

# 시선 정보 형태 변환
gzs_data = within_gzs_data.reshape(-1, 2)  # 샘플 수, 시선 정보 차원

# 데이터셋 분할 (예시: 80% 훈련, 10% 검증, 10% 테스트)
total_samples = normalized_img_data.shape[0]
train_samples = int(total_samples * 0.9)
# val_samples = int(total_samples * 0.1)

train_id = id_one_hot[:train_samples]
train_img = normalized_img_data[:train_samples]
train_hps = hps_data[:train_samples]
train_gzs = gzs_data[:train_samples]

# val_id = id_one_hot[train_samples:train_samples + val_samples]
# val_img = normalized_img_data[train_samples:train_samples + val_samples]
# val_hps = hps_data[train_samples:train_samples + val_samples]
# val_gzs = gzs_data[train_samples:train_samples + val_samples]

# test_id = id_one_hot[train_samples + val_samples:]
# test_img = normalized_img_data[train_samples + val_samples:]
# test_hps = hps_data[train_samples + val_samples:]
# test_gzs = gzs_data[train_samples + val_samples:]

test_id = id_one_hot[train_samples:]
test_img = normalized_img_data[train_samples:]
test_hps = hps_data[train_samples:]
test_gzs = gzs_data[train_samples:]

# 학습 데이터, 검증 데이터, 테스트 데이터 준비
train_data = ((train_img, train_hps, train_id), train_gzs)
# val_data = ((val_img, val_hps, val_id), val_gzs)
test_data = ((test_img, test_hps, test_id), test_gzs)

# 데이터 shape 출력
print("Train data shape:")
print("  Clusters:", train_id.shape)
print("  Images:", train_img.shape)
print("  Head:", train_hps.shape)
print("  Gazes:", train_gzs.shape)

# print("Validation data shape:")
# print("  Clusters:", val_id.shape)
# print("  Images:", val_img.shape)
# print("  Head:", val_hps.shape)
# print("  Gazes:", val_gzs.shape)

print("Test data shape:")
print("  Clusters:", test_id.shape)
print("  Images:", test_img.shape)
print("  Head:", test_hps.shape)
print("  Gazes:", test_gzs.shape)

# Model Architecture

## Image Regressor Class

In [ ]:
class ImageRegressor(keras.Model):
    def __init__(self, name='regressor', **kwargs):
        super(ImageRegressor, self).__init__(name=name, **kwargs)
        
        # 36 x 60
        self.conv0 = keras.layers.Conv2D(64, 3, padding='same', name='conv0')
        self.bn0 = keras.layers.BatchNormalization(name='bn0')
        self.act0 = keras.layers.ELU(name='act0')
        self.maxpool0 = keras.layers.MaxPool2D(name='maxpool0')
        # 18 x 30        
        self.conv1 = keras.layers.Conv2D(128, 3, padding='same', name='conv1')
        self.dropout1 = keras.layers.Dropout(0.5, name='dropout1')
        self.bn1 = keras.layers.BatchNormalization(name='bn1')
        self.act1 = keras.layers.ELU(name='act1')
        self.maxpool1 = keras.layers.MaxPool2D(name='maxpool1')
        # 9 x 15
        self.conv2 = keras.layers.Conv2D(128, 3, padding='same', name='conv2')
        self.bn2 = keras.layers.BatchNormalization(name='bn2')
        self.act2 = keras.layers.ELU(name='act2')
        self.maxpool2 = keras.layers.MaxPool2D(name='maxpool2')
        # 5 x 8
        self.conv3 = keras.layers.Conv2D(256, 3, padding='same', name='conv3')
        self.dropout3 = keras.layers.Dropout(0.5, name='dropout3')
        self.bn3 = keras.layers.BatchNormalization(name='bn3')
        self.act3 = keras.layers.ELU(name='act3')
        self.maxpool3 = keras.layers.MaxPool2D(name='maxpool3')
        # 3 x 4
        self.conv4 = keras.layers.Conv2D(256, 3, padding='same', name='conv4')
        self.bn4 = keras.layers.BatchNormalization(name='bn4')
        self.act4 = keras.layers.ELU(name='act4')
        self.maxpool4 = keras.layers.MaxPool2D(name='maxpool4')
        # 2 x 2
        self.conv5 = keras.layers.Conv2D(512, 3, padding='same', name='conv5')
        self.dropout5 = keras.layers.Dropout(0.5, name='dropout5')
        self.bn5 = keras.layers.BatchNormalization(name='bn5')
        self.act5 = keras.layers.ELU(name='act5')
        self.maxpool5 = keras.layers.MaxPool2D(padding='same', name='maxpool5')
        # # 1 x 1
        self.conv6 = keras.layers.Conv2D(512, 3, padding='same', name='conv6')
        self.act6 = keras.layers.ELU(name='act6')
        self.flatten = keras.layers.Flatten(name='flatten')
        self.dense = keras.layers.Dense(256, name='dense', activation='elu',
                                        kernel_regularizer=keras.regularizers.L1L2(l1=0.01))
        self.out = keras.layers.Dense(2, name='output')
        
    def call(self, inputs, return_layer_activations=False):
        images, head_poses = inputs
        
        c0 = self.conv0(images)
        c0 = self.bn0(c0)
        c0 = self.act0(c0)
        
        c1 = self.maxpool0(c0)
        c1 = self.conv1(c1)
        c1 = self.dropout1(c1)
        c1 = self.bn1(c1)
        c1 = self.act1(c1)
        
        c2 = self.maxpool1(c1)
        c2 = self.conv2(c2)
        c2 = self.bn2(c2)
        c2 = self.act2(c2)
        
        c3 = self.maxpool2(c2)
        c3 = self.conv3(c3)
        c3 = self.dropout3(c3)
        c3 = self.bn3(c3)
        c3 = self.act3(c3)
        
        c4 = self.maxpool3(c3)
        c4 = self.conv4(c4)
        c4 = self.bn4(c4)
        c4 = self.act4(c4)
        
        c5 = self.maxpool4(c4)
        c5 = self.conv5(c5)
        c5 = self.dropout5(c5)
        c5 = self.bn5(c5)
        c5 = self.act5(c5)
        
        c6 = self.maxpool5(c5)
        c6 = self.conv6(c6)
        c6 = self.act6(c6)
        h = self.flatten(c6)
        h = keras.layers.Concatenate()([h, head_poses])
        h = self.dense(h)
        y = self.out(h)
        if return_layer_activations:
            return c0, c1, c2, c3, c4, c5, c6, h, y
        else:
            return y

## Adversarial Regressor Class

In [ ]:
class AdversarialRegressor(keras.Model):
    def __init__(self, n_clusters, name='adversary', **kwargs):   
        super(AdversarialRegressor, self).__init__(name=name, **kwargs)
        self.n_clusters = n_clusters
        # 36 x 60
        self.conv0 = keras.layers.Conv2D(64, 3, padding='same', name='conv0')
        self.bn0 = keras.layers.BatchNormalization(name='bn0')
        self.act0 = keras.layers.ELU(name='act0')
        self.maxpool0 = keras.layers.MaxPool2D(name='maxpool0')
        # 18 x 30        
        self.conv1 = keras.layers.Conv2D(64, 3, padding='same', name='conv1')
        self.dropout1 = keras.layers.Dropout(0.5, name='dropout1')
        self.bn1 = keras.layers.BatchNormalization(name='bn1')
        self.act1 = keras.layers.ELU(name='act1')
        self.maxpool1 = keras.layers.MaxPool2D(name='maxpool1')
        # 9 x 15
        self.conv2 = keras.layers.Conv2D(128, 3, padding='same', name='conv2')
        self.bn2 = keras.layers.BatchNormalization(name='bn2')
        self.act2 = keras.layers.ELU(name='act2')
        self.maxpool2 = keras.layers.MaxPool2D(name='maxpool2')
        # 5 x 8
        self.conv3 = keras.layers.Conv2D(128, 3, padding='same', name='conv3')
        self.dropout3 = keras.layers.Dropout(0.5, name='dropout3')
        self.bn3 = keras.layers.BatchNormalization(name='bn3')
        self.act3 = keras.layers.ELU(name='act3')
        self.maxpool3 = keras.layers.MaxPool2D(name='maxpool3')
        # 3 x 4
        self.conv4 = keras.layers.Conv2D(256, 3, padding='same', name='conv4')
        self.bn4 = keras.layers.BatchNormalization(name='bn4')
        self.act4 = keras.layers.ELU(name='act4')
        self.maxpool4 = keras.layers.MaxPool2D(name='maxpool4')
        # 2 x 2
        self.conv5 = keras.layers.Conv2D(256, 3, padding='same', name='conv5')
        self.dropout5 = keras.layers.Dropout(0.5, name='dropout5')
        self.bn5 = keras.layers.BatchNormalization(name='bn5')
        self.act5 = keras.layers.ELU(name='act5')
        self.maxpool5 = keras.layers.MaxPool2D(padding='same', name='maxpool5')
        # # 1 x 1
        self.conv6 = keras.layers.Conv2D(512, 3, padding='same', name='conv6')
        self.act6 = keras.layers.ELU(name='act6')
        self.flatten = keras.layers.Flatten(name='flatten')
        
        self.dense = keras.layers.Dense(256, name='dense', activation='elu')
        self.out = keras.layers.Dense(n_clusters, name='output', activation='softmax')
        
    def call(self, inputs):
        # Fixed Effect Subnetwork의 Feature map
        c0, c1, c2, c3, c4, c5, c6, h = inputs
        
        x = self.conv0(c0)
        x = self.bn0(x)
        x = self.act0(x)
        x = self.maxpool0(x)
        
        x = keras.layers.Concatenate()([x, c1])
        x = self.conv1(x)
        x = self.dropout1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.maxpool1(x)
        
        x = keras.layers.Concatenate()([x, c2])
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.maxpool2(x)
        
        x = keras.layers.Concatenate()([x, c3])
        x = self.conv3(x)
        x = self.dropout3(x)
        x = self.bn3(x)
        x = self.act3(x)
        x = self.maxpool3(x)
        
        x = keras.layers.Concatenate()([x, c4])
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.act4(x)
        x = self.maxpool4(x)
        
        x = keras.layers.Concatenate()([x, c5])
        x = self.conv5(x)
        x = self.dropout5(x)
        x = self.bn5(x)
        x = self.act5(x)
        
        x = self.maxpool5(x)

        x = self.conv6(x)
        x = self.act6(x)
        x = self.flatten(x)
        x = keras.layers.Concatenate()([x, h])
        x = self.dense(x)
        x = self.out(x)
        return x

## Random Effects Class

In [ ]:
def make_prior_fn(df, scale):
    def prior_fn():
        return tfd.StudentT(df=df, loc=0.0, scale=scale)
    return prior_fn

def make_posterior_fn(df, loc_init_scale, scale_init_min, scale_init_range):
    def posterior_fn(units):
        loc_initializer = tf.random_normal_initializer(mean=0.0, stddev=loc_init_scale)
        scale_initializer = tf.random_uniform_initializer(minval=scale_init_min, maxval=scale_init_min + scale_init_range)

        loc = tf.Variable(initial_value=loc_initializer(shape=(units,)), name="loc", dtype=tf.float32)
        scale = tf.Variable(initial_value=scale_initializer(shape=(units,)), name="scale", dtype=tf.float32)

        return tfd.StudentT(df=df, loc=loc, scale=tf.nn.softplus(scale))
    return posterior_fn

def kl_divergence_student_t(posterior, prior, num_samples=1000, seed=None):
    """Calculate the KL divergence between two Student's T distributions using Monte Carlo approximation."""
    samples = posterior.sample(num_samples, seed=seed)  # Sample from the posterior
    log_posterior_prob = posterior.log_prob(samples)  # Log probability under the posterior
    log_prior_prob = prior.log_prob(samples)  # Log probability under the prior
    
    # Monte Carlo approximation of the KL divergence
    kl_div = tf.reduce_mean(log_posterior_prob - log_prior_prob)
    return kl_div

class RandomEffects(keras.layers.Layer):
    def __init__(self, 
                 units, 
                 df,
                 post_loc_init_scale, 
                 post_scale_init_min, 
                 post_scale_init_range,
                 prior_scale,
                 kl_weight=0.001, 
                 l1_weight=None, 
                 name=None):

        super(RandomEffects, self).__init__(name=name)
        self.kl_weight = kl_weight
        self.l1_weight = l1_weight
        self.units = units

        self.posterior = make_posterior_fn(df, post_loc_init_scale, post_scale_init_min, post_scale_init_range)(units)
        self.prior = make_prior_fn(df, prior_scale)()

    def call(self, inputs, training=None):
        inputs = tf.cast(inputs, dtype=tf.float32)  # 데이터 타입을 float32로 변경
        inputs = tf.expand_dims(inputs, axis=-1)  # inputs의 차원을 확장
        if training:
            # Shape of `samples`: [batch_size, units]
            samples = self.posterior.sample(sample_shape=(tf.shape(inputs)[0],))
            # Ensure 'samples' can be broadcasted with 'inputs'
            # Assuming `inputs` shape is [batch_size, input_dim], we need to align 'samples' along that dimension
            samples = tf.reshape(samples, [tf.shape(inputs)[0], 1, self.units])
            outputs = inputs * samples  # Broadcast multiplication

            kl_div = kl_divergence_student_t(self.posterior, self.prior, num_samples=1000, seed=42)
            self.add_loss(self.kl_weight * tf.reduce_sum(kl_div))
        else:
            # Use the mean of the posterior distribution as a deterministic output
            mean_samples = self.posterior.mean()
            mean_samples = tf.reshape(mean_samples, [1, 1, self.units])
            outputs = inputs * mean_samples  # Broadcast multiplication

        if self.l1_weight:
            self.add_loss(self.l1_weight * tf.reduce_sum(tf.abs(self.posterior.mean())))

        return outputs

## Domain Adversarial Image Regressor Class

In [ ]:
class DomainAdversarialImageRegressor(keras.Model):

    def __init__(self, n_clusters, name='da_regressor', **kwargs):  
        super(DomainAdversarialImageRegressor, self).__init__(name=name, **kwargs)
        self.regressor = ImageRegressor()
        self.adversary = AdversarialRegressor(n_clusters)
        self.n_clusters = n_clusters
        
    def call(self, inputs):
        x, hps, z = inputs
        y_pred = self.regressor((x,hps))
        
        return y_pred
    
    def compile(self,
                loss_regressor=keras.losses.MeanAbsoluteError(),
                loss_adversary=keras.losses.CategoricalCrossentropy(),
                loss_regressor_weight=1.0,
                loss_gen_weight=0.1,
                metric_regressor=keras.metrics.MeanAbsoluteError(),
                opt_regressor=keras.optimizers.Nadam(learning_rate=0.0001),
                opt_adversary=keras.optimizers.Nadam(learning_rate=0.0001)
                ):
      
        super().compile()
        
        self.loss_regressor = loss_regressor
        self.loss_adversary = loss_adversary
        self.loss_regressor_weight = loss_regressor_weight
        self.loss_gen_weight = loss_gen_weight
        self.metric_regressor = metric_regressor
        self.opt_regressor = opt_regressor
        self.opt_adversary = opt_adversary
        
        self.loss_regressor_tracker = keras.metrics.Mean(name='reg_loss')
        self.loss_gen_tracker = keras.metrics.Mean(name='gen_loss')
        self.loss_adversary_tracker = keras.metrics.Mean(name='adv_loss')
        self.loss_total_tracker = keras.metrics.Mean(name='total_loss')
        
    @property
    def metrics(self):
        return [self.loss_regressor_tracker,
                self.loss_gen_tracker,
                self.loss_adversary_tracker,
                self.loss_total_tracker,
                self.metric_regressor]
        
    def _compute_update_loss(self, loss_reg, loss_gen):
        '''Compute total loss and update loss running means'''
        self.loss_regressor_tracker.update_state(loss_reg)
        self.loss_gen_tracker.update_state(loss_gen)
        
        loss_total = (self.loss_regressor_weight * loss_reg) + (self.loss_gen_weight * loss_gen)
        self.loss_total_tracker.update_state(loss_total)
        
        return loss_total
    
    def train_step(self, data):
        if len(data) == 3:
            (images, headpose, clusters), labels, sample_weights = data
        else:
            (images, headpose, clusters), labels = data
            sample_weights = None
            
        # train adversary
        with tf.GradientTape() as gt:
            reg_outs = self.regressor((images, headpose), return_layer_activations=True)
            layer_activations = reg_outs[:-1]
            clusters_pred = self.adversary(layer_activations)
            loss_adv = self.loss_adversary(clusters, clusters_pred, sample_weight=sample_weights)

        grads_adv = gt.gradient(loss_adv, self.adversary.trainable_weights)
        self.opt_adversary.apply_gradients(zip(grads_adv, self.adversary.trainable_weights))
        self.loss_adversary_tracker.update_state(loss_adv)
        
        # train main regressor
        with tf.GradientTape() as gt2:
            reg_outs = self.regressor((images, headpose), return_layer_activations=True)
            y_pred = reg_outs[-1]
            loss_reg = self.loss_regressor(labels, y_pred, sample_weight=sample_weights)
                        
            layer_activations = reg_outs[:-1]
            clusters_pred = self.adversary(layer_activations)
            loss_gen = self.loss_adversary(clusters, clusters_pred, sample_weight=sample_weights)
            
            loss_total = self._compute_update_loss(loss_reg, loss_gen)
            
        grads = gt2.gradient(loss_total, self.regressor.trainable_weights)
        self.opt_regressor.apply_gradients(zip(grads, self.regressor.trainable_weights))
        
        self.metric_regressor.update_state(labels, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        (images, headpose, clusters), labels = data
        
        # Z Predict
        reg_outs = self((images, headpose, clusters), training=False, return_layer_activations=True)
        layer_activations = reg_outs[:-1]
        
        # 클러스터 인덱스 추출
        cluster_indices = tf.argmax(clusters, axis=1)

        # Check if cluster IDs are known
        is_known_cluster = tf.vectorized_map(lambda x: tf.reduce_any(tf.equal(x, self.cluster_list)), cluster_indices)
        
        # Calculate adversary output once and reuse
        adversary_output = self.adversary(layer_activations)

        # Use known or predicted clusters
        new_clusters = tf.where(tf.expand_dims(is_known_cluster, axis=1), clusters, adversary_output)

        # Main Regression with potentially new clusters
        reg_outs = self((images, headpose, new_clusters), training=False, return_layer_activations=True)
        y_pred = reg_outs[-1]
        
        loss_reg = self.loss_regressor(labels, y_pred)
        loss_gen = self.loss_adversary(new_clusters, adversary_output)
        
        _ = self._compute_update_loss(loss_reg, loss_gen)
        
        self.metric_regressor.update_state(labels, y_pred)
        return {m.name: m.result() for m in self.metrics}

## Mixed Effects Regressor

In [ ]:
class MixedEffectsRegressor(DomainAdversarialImageRegressor):
    def __init__(self, 
                 cluster_list=[],
                 slope_post_init_scale=0.1,
                 intercept_post_init_scale=0.1,
                 slope_scale=0.25,
                 intercept_scale=0.25,
                 kl_weight=1e-3,
                 df=10,
                 name='me_regressor', **kwargs):

        self.cluster_list = tf.constant([int(cid[1:]) for cid in cluster_list], dtype=tf.int64)
        n_clusters = len(self.cluster_list)
        super(MixedEffectsRegressor, self).__init__(n_clusters, name=name, **kwargs)
                
        self.re_slopes = RandomEffects(units=256,
                                       df=df,
                                       post_loc_init_scale=slope_post_init_scale,
                                       post_scale_init_min=0.01, 
                                       post_scale_init_range=0.02,
                                       prior_scale=slope_scale,
                                       kl_weight=kl_weight,
                                       name='re_slopes')
        
        self.re_intercept = RandomEffects(units=1,
                                          df=df,
                                          post_loc_init_scale=intercept_post_init_scale,
                                          post_scale_init_min=0.01, 
                                          post_scale_init_range=0.02,
                                          prior_scale=intercept_scale,
                                          kl_weight=kl_weight,
                                          name='re_intercept')
        
    def call(self, inputs, training, return_layer_activations=False):
        x, hps, z = inputs
        
        if x.shape[-1] != 1:
            x = tf.expand_dims(x, -1)

        c0 = self.regressor.conv0(x)
        c0 = self.regressor.bn0(c0)
        c0 = self.regressor.act0(c0)
        
        c1 = self.regressor.maxpool0(c0)
        c1 = self.regressor.conv1(c1)
        c1 = self.regressor.dropout1(c1)
        c1 = self.regressor.bn1(c1)
        c1 = self.regressor.act1(c1)
        
        c2 = self.regressor.maxpool1(c1)
        c2 = self.regressor.conv2(c2)
        c2 = self.regressor.bn2(c2)
        c2 = self.regressor.act2(c2)
        
        c3 = self.regressor.maxpool2(c2)
        c3 = self.regressor.conv3(c3)
        c3 = self.regressor.dropout3(c3)
        c3 = self.regressor.bn3(c3)
        c3 = self.regressor.act3(c3)
        
        c4 = self.regressor.maxpool3(c3)
        c4 = self.regressor.conv4(c4)
        c4 = self.regressor.bn4(c4)
        c4 = self.regressor.act4(c4)
        
        c5 = self.regressor.maxpool4(c4)
        c5 = self.regressor.conv5(c5)
        c5 = self.regressor.dropout5(c5)
        c5 = self.regressor.bn5(c5)
        c5 = self.regressor.act5(c5)
        
        c6 = self.regressor.maxpool5(c5)
        c6 = self.regressor.conv6(c6)
        c6 = self.regressor.act6(c6)
        h = self.regressor.flatten(c6)
        h = tf.concat([h, hps], axis=-1)
        h = self.regressor.dense(h)
        
        slopes = self.re_slopes(z, training=training)
        intercepts = self.re_intercept(z, training=training)
        if len(slopes.shape) == 3:
            slopes = tf.reduce_mean(slopes, axis=1)
        if len(intercepts.shape) == 3:
            intercepts = tf.reduce_mean(intercepts, axis=1)
        y = self.regressor.out(h * (1 + slopes))

        # Apply intercepts
        y = y + intercepts
        
        if return_layer_activations:
            return c0, c1, c2, c3, c4, c5, c6, h, y
        else:
            return y
    
    def compile(self,
            loss_regressor=keras.losses.MeanAbsoluteError(),
            loss_adversary=keras.losses.CategoricalCrossentropy(),
            loss_regressor_weight=1.0,
            loss_gen_weight=0.1,
            metric_regressor=keras.metrics.MeanAbsoluteError(),
            opt_regressor=keras.optimizers.Nadam(learning_rate=0.0001),
            opt_adversary=keras.optimizers.Nadam(learning_rate=0.0001)
            ):
        
        super(MixedEffectsRegressor, self).compile(loss_regressor=loss_regressor,
                                                    loss_adversary=loss_adversary,
                                                    loss_regressor_weight=loss_regressor_weight,
                                                    loss_gen_weight=loss_gen_weight,
                                                    metric_regressor=metric_regressor,
                                                    opt_regressor=opt_regressor,
                                                    opt_adversary=opt_adversary)
        self.loss_kld_tracker = tf.keras.metrics.Mean(name='kld')
        
    @property
    def metrics(self):
        return [self.loss_regressor_tracker,
                self.loss_gen_tracker,
                self.loss_adversary_tracker,
                self.loss_kld_tracker,
                self.loss_total_tracker,
                self.metric_regressor]
        
    def _compute_update_loss(self, loss_reg, loss_gen, training=True):
        '''Compute total loss and update loss running means'''
        self.loss_regressor_tracker.update_state(loss_reg)
        self.loss_gen_tracker.update_state(loss_gen)
        if training:
            kld = tf.reduce_sum(self.re_slopes.losses) + tf.reduce_sum(self.re_intercept.losses)
            self.loss_kld_tracker.update_state(kld)
        else:
            kld = 0
        loss_total = (self.loss_regressor_weight * loss_reg) \
            + (self.loss_gen_weight * loss_gen) \
            + kld
        self.loss_total_tracker.update_state(loss_total)
        
        return loss_total
        
    
    def train_step(self, data):
        (images, headpose, clusters), labels = data
        sample_weights = None

        # train adversary
        with tf.GradientTape() as gt:
            reg_outs = self((images, headpose, clusters), training=True, return_layer_activations=True)
            layer_activations = reg_outs[:-1]
            clusters_pred = self.adversary(layer_activations)
            loss_adv = self.loss_adversary(clusters, clusters_pred, sample_weight=sample_weights)

        grads_adv = gt.gradient(loss_adv, self.adversary.trainable_weights)
        self.opt_adversary.apply_gradients(zip(grads_adv, self.adversary.trainable_weights))
        self.loss_adversary_tracker.update_state(loss_adv)

        # train main regressor
        with tf.GradientTape() as gt2:
            reg_outs = self((images, headpose, clusters), training=True, return_layer_activations=True)
            y_pred = reg_outs[-1]
            # 각 출력에 대한 손실을 계산하고 합산
            total_loss_reg = 0
            for i in range(y_pred.shape[1]):  # y_pred의 출력 개수만큼 반복
                if sample_weights is not None:
                    total_loss_reg += self.loss_regressor(labels[:, i], y_pred[:, i], sample_weight=sample_weights[:, i])
                else:
                    total_loss_reg += self.loss_regressor(labels[:, i], y_pred[:, i])

            layer_activations = reg_outs[:-1]
            clusters_pred = self.adversary(layer_activations)
            loss_gen = self.loss_adversary(clusters, clusters_pred, sample_weight=sample_weights)

            loss_total = self._compute_update_loss(total_loss_reg, loss_gen)

        grads = gt2.gradient(loss_total, self.regressor.trainable_weights)
        self.opt_regressor.apply_gradients(zip(grads, self.regressor.trainable_weights))

        self.metric_regressor.update_state(labels, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        (images, headpose, clusters), labels = data
        
        # Z Predict
        reg_outs = self((images, headpose, clusters), training=False, return_layer_activations=True)
        layer_activations = reg_outs[:-1]
        
        # 클러스터 인덱스 추출
        cluster_indices = tf.argmax(clusters, axis=1)

        # Check if cluster IDs are known
        is_known_cluster = tf.vectorized_map(lambda x: tf.reduce_any(tf.equal(x, self.cluster_list)), cluster_indices)
        
        # Calculate adversary output once and reuse
        adversary_output = self.adversary(layer_activations)

        # Use known or predicted clusters
        new_clusters = tf.where(tf.expand_dims(is_known_cluster, axis=1), clusters, adversary_output)

        # Main Regression with potentially new clusters
        reg_outs = self((images, headpose, new_clusters), training=False, return_layer_activations=True)
        y_pred = reg_outs[-1]
        
        loss_reg = self.loss_regressor(labels, y_pred)
        loss_gen = self.loss_adversary(new_clusters, adversary_output)
        
        _ = self._compute_update_loss(loss_reg, loss_gen)
        
        self.metric_regressor.update_state(labels, y_pred)
        return {m.name: m.result() for m in self.metrics}

# Training

In [ ]:
def mae_keras(predictedGaze, groundtruthGaze, is_3d=False, deg=False):
    '''
    Calculate Mean Angular Error using TensorFlow.
    
    Args:
    predictedGaze (tf.Tensor): Predicted gaze vectors.
    groundtruthGaze (tf.Tensor): Ground truth gaze vectors.
    is_3d (bool): Flag indicating whether the input vectors are 3D. Default is False.
    deg (bool): Flag indicating whether the spherical coordinates are in degrees. Default is False.
    
    Returns:
    tf.Tensor: Mean angular error.
    '''
    Gaze_1 = tf.cast(predictedGaze, dtype=tf.float32)
    Gaze_2 = tf.cast(groundtruthGaze, dtype=tf.float32)
    
    if not is_3d:
        Gaze_1 = convert_to_xyz_tf(Gaze_1, deg)
        Gaze_2 = convert_to_xyz_tf(Gaze_2, deg)

    Gaze_1 = Gaze_1 / tf.norm(Gaze_1, axis=1, keepdims=True)
    Gaze_2 = Gaze_2 / tf.norm(Gaze_2, axis=1, keepdims=True)

    cos_val = tf.reduce_sum(Gaze_1 * Gaze_2, axis=1)
    cos_val = tf.clip_by_value(cos_val, -1, 1)  # Ensure cos_val is within the valid range for arccos

    angle_val = tf.acos(cos_val) * 180 / tf.constant(np.pi)

    return tf.reduce_mean(angle_val)

def convert_to_xyz_tf(spherical, deg=False):
    if deg:
        spherical = spherical * tf.constant(np.pi) / 180

    # Create xyz tensor similarly as above but using TensorFlow operations.
    xyz = tf.zeros((spherical.shape[0], 3), dtype=tf.float32)
    xyz = tf.tensor_scatter_nd_update(
        xyz, [[i, 0] for i in range(spherical.shape[0])],
        -tf.cos(spherical[:, 0]) * tf.sin(spherical[:, 1]))
    xyz = tf.tensor_scatter_nd_update(
        xyz, [[i, 1] for i in range(spherical.shape[0])],
        -tf.sin(spherical[:, 0]))
    xyz = tf.tensor_scatter_nd_update(
        xyz, [[i, 2] for i in range(spherical.shape[0])],
        -tf.cos(spherical[:, 0]) * tf.cos(spherical[:, 1]))

    xyz = xyz / tf.norm(xyz, axis=1, keepdims=True)

    return xyz

In [ ]:
# 모델 생성
df = 3
model = MixedEffectsRegressor(cluster_list = unique_ids, df = df)

def t_loss(k):
    def loss(y_true, y_pred, sample_weight=None):
        error = y_true - y_pred
        squared_error = tf.square(error)
        scaled_error = tf.math.log(k + squared_error)
        if sample_weight is not None:
            scaled_error = tf.multiply(sample_weight, scaled_error)
        return tf.reduce_mean(scaled_error)
    return loss

# 모델 컴파일
model.compile(loss_regressor=t_loss(df),
              loss_adversary=keras.losses.CategoricalCrossentropy(),
              loss_regressor_weight=7.5,
              loss_gen_weight=0.5,
              metric_regressor=tf.keras.metrics.MeanAbsoluteError(),
              opt_regressor=tf.keras.optimizers.Adam(learning_rate=0.0001),
              opt_adversary=tf.keras.optimizers.Adam(learning_rate=0.0001))

# 콜백 설정
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# 모델 학습
history = model.fit(
    x=train_data[0],  # 입력 데이터: 이미지와 클러스터 정보
    y=train_data[1],  # 타겟 데이터: 시선 벡터
    epochs=350,
    batch_size=32,
    # validation_data=val_data,  # 검증 데이터
    # callbacks=[early_stopping, reduce_lr]
)

In [ ]:
pred_gzs = model.predict((test_img, test_hps, test_id))

In [ ]:
print(f"Predict Sample : {pred_gzs[-5:]}")
print(f"True Sample : {test_gzs[-5:]}")

In [ ]:
# MAE 계산
error = mae_keras(pred_gzs, test_gzs)
print("Mean Angular Error:", error.numpy(), "degrees")